### Summary
In this notebook, I bring in data from postgres via AWS, clean it, and perform some EDA to prepare for modeling.

Data were gathered by the reporting team at APM Reports for the podcast "In The Dark"

In [2]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import pickle

In [3]:
params = {
    'host': #your host  ,
    'user': 'ubuntu',
    'port': 5432,
    'database' : 'jury_selection'
}

In [4]:
connection_string = f'postgres://ubuntu:{params["host"]}@{params["host"]}:{params["port"]}/{params["database"]}'

engine = create_engine(connection_string)

### Juror Data

In [202]:
jurors = pd.read_sql('SELECT * FROM jurors', engine)

In [209]:
jurors.head()

,juror_id,trial,trial_id,race,gender,race_source,gender_source,struck_by,strike_eligibility
0,15077,1998-6316--Bobby Rice,306,Unknown,Unknown,No info,Jury strike sheet,Juror not struck,n/a
1,15076,1998-6316--Bobby Rice,306,Unknown,Unknown,No info,Jury strike sheet,Juror not struck,n/a
2,15075,1998-6316--Bobby Rice,306,Unknown,Unknown,No info,Jury strike sheet,Juror not struck,n/a
3,15074,1998-6316--Bobby Rice,306,Unknown,Unknown,No info,Jury strike sheet,Juror not struck,n/a
4,15073,1998-6316--Bobby Rice,306,Unknown,Unknown,No info,Jury strike sheet,Juror not struck,n/a


In [210]:
jurors.struck_by.value_counts()

Juror chosen to serve on jury    3640
Juror not struck                 3436
Struck by the defense            2301
Struck for cause                 2042
Struck by the state              1852
Struck without notation           571
Juror excused/absent              371
Juror chosen as alternate         370
Unknown                           291
Name: struck_by, dtype: int64

In [211]:
jurors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14874 entries, 0 to 14873
Data columns (total 9 columns):
juror_id              14874 non-null int64
trial                 14874 non-null object
trial_id              14874 non-null int64
race                  14874 non-null object
gender                14874 non-null object
race_source           14874 non-null object
gender_source         14874 non-null object
struck_by             14874 non-null object
strike_eligibility    14874 non-null object
dtypes: int64(2), object(7)
memory usage: 1.1+ MB


In [212]:
jurors = jurors[['juror_id', 'trial_id', 'race', 'gender', 'struck_by', 'strike_eligibility']]

In [213]:
jurors.race.replace(to_replace=dict(Unknown=np.nan), inplace=True)

In [ ]:
jurors = jurors.dropna()

Removed over 4,700 jurors for whom there was no race information. 

In [215]:
jurors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10122 entries, 45 to 14873
Data columns (total 6 columns):
juror_id              10122 non-null int64
trial_id              10122 non-null int64
race                  10122 non-null object
gender                10122 non-null object
struck_by             10122 non-null object
strike_eligibility    10122 non-null object
dtypes: int64(2), object(4)
memory usage: 553.5+ KB


In [216]:
jurors.gender.value_counts()

Female     5441
Male       4032
Unknown     649
Name: gender, dtype: int64

In [217]:
jurors.strike_eligibility.value_counts()

n/a                       4700
Both State and Defense    3504
State                     1627
Defense                    155
Neither                    136
Name: strike_eligibility, dtype: int64

In [218]:
jurors.head() 

,juror_id,trial_id,race,gender,struck_by,strike_eligibility
45,15032,255,Black,Male,Struck for cause,n/a
46,15031,255,White,Female,Struck for cause,n/a
47,15030,255,White,Male,Struck for cause,n/a
48,15029,255,White,Female,Struck for cause,n/a
49,15028,255,White,Male,Struck for cause,n/a


In [241]:
jurors= jurors.replace({'race': {'Black': 1, 'White': 0}})

In [242]:
jurors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10122 entries, 45 to 14873
Data columns (total 6 columns):
juror_id              10122 non-null int64
trial_id              10122 non-null int64
race                  10122 non-null object
gender                10122 non-null object
struck_by             10122 non-null object
strike_eligibility    10122 non-null object
dtypes: int64(2), object(4)
memory usage: 553.5+ KB


### Trial Data

In [197]:
trials = pd.read_sql('SELECT * FROM trials', engine)

In [198]:
trials.head()

,trial_id,defendant_name,cause_number,state_strikes,defense_strikes,county,defendent_race,second_defendent_race,third_defendent_race,fourth_defendent_race,...,offense_code_5,offense_title_5,offense_code_6,offense_title_6,more_than_six,verdict,case_appealed,batson_claim_by_defense,batson_claim_by_state,voir_dire_present
0,306,Bobby Rice,1998-6316,True,True,Winston,Black,n/a,n/a,n/a,...,None,None,None,None,False,Guilty on at least one offense,True,False,False,True
1,305,Edwin Hart Turner,1996-3176,False,True,Carroll,White,n/a,n/a,n/a,...,None,None,None,None,False,Guilty on at least one offense,True,False,False,True
2,304,Sheila Jackson,1992-4407,False,False,Winston,Black,n/a,n/a,n/a,...,None,None,None,None,False,Guilty on at least one offense,True,False,False,True
3,303,Christopher Fair,2005-0007,False,True,Choctaw,Black,n/a,n/a,n/a,...,None,None,None,None,False,Guilty on at least one offense,True,False,False,True
4,302,Richard Dickerson,2006-0075,False,False,Grenada,Black,n/a,n/a,n/a,...,None,None,None,None,False,Unknown,True,False,False,False


In [200]:
trials.state_strikes.value_counts()

False    214
True      91
Name: state_strikes, dtype: int64

In [201]:
trials.defense_strikes.value_counts()

True     180
False    125
Name: defense_strikes, dtype: int64

In [37]:
trials.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Data columns (total 38 columns):
trial_id                       305 non-null int64
defendant_name                 305 non-null object
cause_number                   305 non-null object
state_strikes                  305 non-null bool
defense_strikes                305 non-null bool
county                         305 non-null object
defendent_race                 305 non-null object
second_defendent_race          305 non-null object
third_defendent_race           305 non-null object
fourth_defendent_race          305 non-null object
more_than_four_defendants      305 non-null bool
judge                          305 non-null object
prosecutor_1                   303 non-null object
prosecutor_2                   189 non-null object
prosecutor_3                   21 non-null object
prosecutors_more_than_three    305 non-null bool
def_attny_1                    304 non-null object
def_attny_2                    38 non-

In [38]:
trials = trials[['trial_id', 'state_strikes', 'defense_strikes', 'defendent_race', 'batson_claim_by_defense',
                'batson_claim_by_state']]

In [40]:
trials.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Data columns (total 6 columns):
trial_id                   305 non-null int64
state_strikes              305 non-null bool
defense_strikes            305 non-null bool
defendent_race             305 non-null object
batson_claim_by_defense    305 non-null bool
batson_claim_by_state      305 non-null bool
dtypes: bool(4), int64(1), object(1)
memory usage: 6.0+ KB


In [41]:
trials['defendent_race'].unique()

array(['Black', 'White', 'Unknown', 'Latino', 'Asian'], dtype=object)

In [42]:
trials = trials[trials.defendent_race != 'Unknown']
trials = trials[trials.defendent_race != 'Latino']
trials = trials[trials.defendent_race != 'Asian']

In [43]:
trials['defendent_race'].unique()

array(['Black', 'White'], dtype=object)

In [76]:
trials_model = trials[['trial_id', 'defendent_race']]

In [44]:
trials.dtypes

trial_id                    int64
state_strikes                bool
defense_strikes              bool
defendent_race             object
batson_claim_by_defense      bool
batson_claim_by_state        bool
dtype: object

### Voir Dire Answers

In [105]:
answers = pd.read_sql('SELECT * FROM voir_dire_answers', engine)

In [46]:
answers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3546 entries, 0 to 3545
Data columns (total 70 columns):
id                      3546 non-null int64
juror_id                3546 non-null int64
trial_id                3546 non-null int64
no_responses            3546 non-null bool
married                 3546 non-null object
children                3546 non-null object
religious               3546 non-null object
education               3546 non-null object
leans_state             3546 non-null bool
leans_defense           3546 non-null bool
leans_ambi              3546 non-null bool
moral_hardship          3546 non-null bool
job_hardship            3546 non-null bool
caretaker               3546 non-null bool
communication           3546 non-null bool
medical                 3546 non-null bool
employed                3546 non-null bool
social                  3546 non-null bool
prior_jury              3546 non-null bool
crime_victim            3546 non-null bool
fam_crime_victim      

In [103]:
answers.head()

,id,juror_id,trial_id,no_responses,married,children,religious,education,leans_state,leans_defense,...,witness_defense,witness_state,prior_info,death_hesitation,no_death,no_life,no_cops,yes_cops,legally_disqualified,witness_ambi
0,489,0,0,True,unknown,unknown,unknown,unknown,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,1,1,False,FALSE,FALSE,unknown,unknown,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2,3,1,False,FALSE,FALSE,unknown,unknown,False,False,...,False,False,False,False,False,False,False,False,False,False
3,3,4,1,False,FALSE,FALSE,unknown,unknown,False,False,...,False,False,False,False,False,False,False,False,False,False
4,4,5,1,False,FALSE,FALSE,unknown,unknown,False,False,...,False,False,False,False,False,False,False,False,False,False


In [106]:
answers = answers.drop(columns = ['notes', 'id'])

### Struck for cause

In [48]:
struck_for_cause = pd.read_sql("""

    SELECT * FROM jurors WHERE struck_by = 'Struck for cause'

    """, engine)

In [49]:
struck_for_cause.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2042 entries, 0 to 2041
Data columns (total 9 columns):
juror_id              2042 non-null int64
trial                 2042 non-null object
trial_id              2042 non-null int64
race                  2042 non-null object
gender                2042 non-null object
race_source           2042 non-null object
gender_source         2042 non-null object
struck_by             2042 non-null object
strike_eligibility    2042 non-null object
dtypes: int64(2), object(7)
memory usage: 143.7+ KB


In [50]:
cause = struck_for_cause.merge(answers, on = 'juror_id')

In [53]:
cause.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 912 entries, 0 to 911
Data columns (total 78 columns):
juror_id                912 non-null int64
trial                   912 non-null object
trial_id_x              912 non-null int64
race                    912 non-null object
gender                  912 non-null object
race_source             912 non-null object
gender_source           912 non-null object
struck_by               912 non-null object
strike_eligibility      912 non-null object
id                      912 non-null int64
trial_id_y              912 non-null int64
no_responses            912 non-null bool
married                 912 non-null object
children                912 non-null object
religious               912 non-null object
education               912 non-null object
leans_state             912 non-null bool
leans_defense           912 non-null bool
leans_ambi              912 non-null bool
moral_hardship          912 non-null bool
job_hardship            912 n

In [193]:
cause.race.value_counts()

Black      488
White      409
Unknown     15
Name: race, dtype: int64

Of the 2042 jurors struck for cause, 912 have documented voir dire answers. Of them, 53.5% are Black and 44.8% are white

In [194]:
cause.gender.value_counts()

Female     536
Male       366
Unknown     10
Name: gender, dtype: int64

### Examine those struck by prosecution

In [54]:
struck_by_pros = pd.read_sql("""

    SELECT * FROM jurors WHERE struck_by = 'Struck by the state'

    """, engine)

In [55]:
struck_by_pros.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1852 entries, 0 to 1851
Data columns (total 9 columns):
juror_id              1852 non-null int64
trial                 1852 non-null object
trial_id              1852 non-null int64
race                  1852 non-null object
gender                1852 non-null object
race_source           1852 non-null object
gender_source         1852 non-null object
struck_by             1852 non-null object
strike_eligibility    1852 non-null object
dtypes: int64(2), object(7)
memory usage: 130.3+ KB


In [56]:
struck_by_pros.race.replace(to_replace=dict(Unknown=np.nan), inplace=True)

In [57]:
struck_by_pros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1852 entries, 0 to 1851
Data columns (total 9 columns):
juror_id              1852 non-null int64
trial                 1852 non-null object
trial_id              1852 non-null int64
race                  1286 non-null object
gender                1852 non-null object
race_source           1852 non-null object
gender_source         1852 non-null object
struck_by             1852 non-null object
strike_eligibility    1852 non-null object
dtypes: int64(2), object(7)
memory usage: 130.3+ KB


In [59]:
struck_by_pros = struck_by_pros.dropna()

In [60]:
struck_by_pros.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1286 entries, 21 to 1851
Data columns (total 9 columns):
juror_id              1286 non-null int64
trial                 1286 non-null object
trial_id              1286 non-null int64
race                  1286 non-null object
gender                1286 non-null object
race_source           1286 non-null object
gender_source         1286 non-null object
struck_by             1286 non-null object
strike_eligibility    1286 non-null object
dtypes: int64(2), object(7)
memory usage: 100.5+ KB


In [61]:
struck_by_pros['race'].unique()

array(['White', 'Black', 'Latino'], dtype=object)

In [64]:
struck_by_pros = struck_by_pros[struck_by_pros.race != 'Latino']

In [65]:
struck_by_pros.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1285 entries, 21 to 1851
Data columns (total 9 columns):
juror_id              1285 non-null int64
trial                 1285 non-null object
trial_id              1285 non-null int64
race                  1285 non-null object
gender                1285 non-null object
race_source           1285 non-null object
gender_source         1285 non-null object
struck_by             1285 non-null object
strike_eligibility    1285 non-null object
dtypes: int64(2), object(7)
memory usage: 100.4+ KB


In [80]:
struck_by_pros.groupby('race').count()

,juror_id,trial,trial_id,gender,race_source,gender_source,struck_by,strike_eligibility
race,,,,,,,,
Black,911,911,911,911,911,911,911,911
White,374,374,374,374,374,374,374,374


The prosecution's strikes were directed toward black people 71% of the time and white people 29% of the time.

In [123]:
struck_by_pros.groupby('gender').count()

,juror_id,trial,trial_id,race,race_source,gender_source,struck_by,strike_eligibility
gender,,,,,,,,
Female,730,730,730,730,730,730,730,730
Male,479,479,479,479,479,479,479,479
Unknown,76,76,76,76,76,76,76,76


### Struck by Defense

In [67]:
struck_by_def = pd.read_sql("""

    SELECT * FROM jurors WHERE struck_by = 'Struck by the defense'

    """, engine)

In [68]:
struck_by_def.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2301 entries, 0 to 2300
Data columns (total 9 columns):
juror_id              2301 non-null int64
trial                 2301 non-null object
trial_id              2301 non-null int64
race                  2301 non-null object
gender                2301 non-null object
race_source           2301 non-null object
gender_source         2301 non-null object
struck_by             2301 non-null object
strike_eligibility    2301 non-null object
dtypes: int64(2), object(7)
memory usage: 161.9+ KB


In [69]:
struck_by_def.race.replace(to_replace=dict(Unknown=np.nan), inplace=True)

In [72]:
struck_by_def = struck_by_def.dropna()

In [73]:
struck_by_def.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1467 entries, 55 to 2300
Data columns (total 9 columns):
juror_id              1467 non-null int64
trial                 1467 non-null object
trial_id              1467 non-null int64
race                  1467 non-null object
gender                1467 non-null object
race_source           1467 non-null object
gender_source         1467 non-null object
struck_by             1467 non-null object
strike_eligibility    1467 non-null object
dtypes: int64(2), object(7)
memory usage: 114.6+ KB


In [71]:
struck_by_def['race'].unique()

array([nan, 'White', 'Black', 'Asian'], dtype=object)

In [74]:
struck_by_def = struck_by_def[struck_by_def.race != 'Asian']

In [75]:
struck_by_def.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1466 entries, 55 to 2300
Data columns (total 9 columns):
juror_id              1466 non-null int64
trial                 1466 non-null object
trial_id              1466 non-null int64
race                  1466 non-null object
gender                1466 non-null object
race_source           1466 non-null object
gender_source         1466 non-null object
struck_by             1466 non-null object
strike_eligibility    1466 non-null object
dtypes: int64(2), object(7)
memory usage: 114.5+ KB


In [81]:
struck_by_def.groupby('race').count()

,juror_id,trial,trial_id,gender,race_source,gender_source,struck_by,strike_eligibility
race,,,,,,,,
Black,128,128,128,128,128,128,128,128
White,1338,1338,1338,1338,1338,1338,1338,1338


The defense used 91% of its peremptory strikes on white people. It's important to note that the pool of jurors they could strike was majority white (get exact #)

In [124]:
struck_by_def.groupby('gender').count()

,juror_id,trial,trial_id,race,race_source,gender_source,struck_by,strike_eligibility
gender,,,,,,,,
Female,715,715,715,715,715,715,715,715
Male,661,661,661,661,661,661,661,661
Unknown,90,90,90,90,90,90,90,90


### On Jury

In [83]:
on_jury = pd.read_sql("""

    SELECT * FROM jurors WHERE struck_by = 'Juror chosen to serve on jury' 
    OR struck_by = 'Juror chosen as alternate'

    """, engine)

In [84]:
on_jury.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4010 entries, 0 to 4009
Data columns (total 9 columns):
juror_id              4010 non-null int64
trial                 4010 non-null object
trial_id              4010 non-null int64
race                  4010 non-null object
gender                4010 non-null object
race_source           4010 non-null object
gender_source         4010 non-null object
struck_by             4010 non-null object
strike_eligibility    4010 non-null object
dtypes: int64(2), object(7)
memory usage: 282.0+ KB


In [85]:
on_jury.race.replace(to_replace=dict(Unknown=np.nan), inplace=True)

In [86]:
on_jury.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4010 entries, 0 to 4009
Data columns (total 9 columns):
juror_id              4010 non-null int64
trial                 4010 non-null object
trial_id              4010 non-null int64
race                  2702 non-null object
gender                4010 non-null object
race_source           4010 non-null object
gender_source         4010 non-null object
struck_by             4010 non-null object
strike_eligibility    4010 non-null object
dtypes: int64(2), object(7)
memory usage: 282.0+ KB


In [87]:
on_jury = on_jury.dropna()

In [88]:
on_jury.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2702 entries, 29 to 4009
Data columns (total 9 columns):
juror_id              2702 non-null int64
trial                 2702 non-null object
trial_id              2702 non-null int64
race                  2702 non-null object
gender                2702 non-null object
race_source           2702 non-null object
gender_source         2702 non-null object
struck_by             2702 non-null object
strike_eligibility    2702 non-null object
dtypes: int64(2), object(7)
memory usage: 211.1+ KB


In [89]:
on_jury['race'].unique()

array(['White', 'Black'], dtype=object)

In [90]:
on_jury.groupby('race').count()

,juror_id,trial,trial_id,gender,race_source,gender_source,struck_by,strike_eligibility
race,,,,,,,,
Black,861,861,861,861,861,861,861,861
White,1841,1841,1841,1841,1841,1841,1841,1841


The jurors that made it on the jury (or served as alternates) were 68% white

In [125]:
on_jury.groupby('gender').count()

,juror_id,trial,trial_id,race,race_source,gender_source,struck_by,strike_eligibility
gender,,,,,,,,
Female,1509,1509,1509,1509,1509,1509,1509,1509
Male,1002,1002,1002,1002,1002,1002,1002,1002
Unknown,191,191,191,191,191,191,191,191


### Aggregate data

In [225]:
model = jurors.merge(answers, on = 'juror_id')

In [226]:
model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3458 entries, 0 to 3457
Data columns (total 73 columns):
juror_id                3458 non-null int64
trial_id_x              3458 non-null int64
race                    3458 non-null object
gender                  3458 non-null object
struck_by               3458 non-null object
strike_eligibility      3458 non-null object
trial_id_y              3458 non-null int64
no_responses            3458 non-null bool
married                 3458 non-null object
children                3458 non-null object
religious               3458 non-null object
education               3458 non-null object
leans_state             3458 non-null bool
leans_defense           3458 non-null bool
leans_ambi              3458 non-null bool
moral_hardship          3458 non-null bool
job_hardship            3458 non-null bool
caretaker               3458 non-null bool
communication           3458 non-null bool
medical                 3458 non-null bool
employed      

In [227]:
model.head()

,juror_id,trial_id_x,race,gender,struck_by,strike_eligibility,trial_id_y,no_responses,married,children,...,witness_defense,witness_state,prior_info,death_hesitation,no_death,no_life,no_cops,yes_cops,legally_disqualified,witness_ambi
0,14522,28,White,Female,Juror excused/absent,n/a,28,False,unknown,unknown,...,False,False,False,False,False,False,False,False,False,False
1,14521,28,White,Male,Struck for cause,n/a,28,False,unknown,unknown,...,False,False,False,False,False,False,False,False,False,False
2,14520,31,White,Male,Struck for cause,n/a,31,False,unknown,unknown,...,False,False,False,False,False,False,False,False,False,False
3,14519,197,White,Female,Juror chosen to serve on jury,Neither,197,True,unknown,unknown,...,False,False,False,False,False,False,False,False,False,False
4,14478,301,Black,Male,Struck for cause,n/a,301,False,unknown,unknown,...,False,False,False,False,False,False,False,False,False,False


In [228]:
model = model.rename(columns = {'trial_id_x': 'trial_id'})

In [229]:
model = model.drop(columns = ['trial_id_y'])

In [230]:
model = model.merge(trials_model, on = 'trial_id')

In [231]:
model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3353 entries, 0 to 3352
Data columns (total 73 columns):
juror_id                3353 non-null int64
trial_id                3353 non-null int64
race                    3353 non-null object
gender                  3353 non-null object
struck_by               3353 non-null object
strike_eligibility      3353 non-null object
no_responses            3353 non-null bool
married                 3353 non-null object
children                3353 non-null object
religious               3353 non-null object
education               3353 non-null object
leans_state             3353 non-null bool
leans_defense           3353 non-null bool
leans_ambi              3353 non-null bool
moral_hardship          3353 non-null bool
job_hardship            3353 non-null bool
caretaker               3353 non-null bool
communication           3353 non-null bool
medical                 3353 non-null bool
employed                3353 non-null bool
social         

In [232]:
model['struck_by_prosecution'] = (model.struck_by == 'Struck by the state').astype(int)

In [233]:
model['married'].unique()

array(['unknown', 'divorced', 'married', 'single',
       'living unmarried with significant other', 'widowed', 'FALSE'],
      dtype=object)

In [234]:
model['married'].value_counts()

,juror_id,trial_id,race,gender,struck_by,strike_eligibility,no_responses,children,religious,education,...,prior_info,death_hesitation,no_death,no_life,no_cops,yes_cops,legally_disqualified,witness_ambi,defendent_race,struck_by_prosecution
married,,,,,,,,,,,,,,,,,,,,,
FALSE,36,36,36,36,36,36,36,36,36,36,...,36,36,36,36,36,36,36,36,36,36
divorced,15,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,15,15,15
living unmarried with significant other,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
married,168,168,168,168,168,168,168,168,168,168,...,168,168,168,168,168,168,168,168,168,168
single,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
unknown,3128,3128,3128,3128,3128,3128,3128,3128,3128,3128,...,3128,3128,3128,3128,3128,3128,3128,3128,3128,3128
widowed,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4


In [116]:
model['children'].unique()

array(['unknown', 'yes', 'no', 'FALSE'], dtype=object)

In [117]:
model['children'].value_counts()

,juror_id,trial_id,race,gender,struck_by,strike_eligibility,no_responses,married,religious,education,...,prior_info,death_hesitation,no_death,no_life,no_cops,yes_cops,legally_disqualified,witness_ambi,defendent_race,struck_by_prosecution
children,,,,,,,,,,,,,,,,,,,,,
FALSE,36,36,36,36,36,36,36,36,36,36,...,36,36,36,36,36,36,36,36,36,36
no,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
unknown,3183,3183,3183,3183,3183,3183,3183,3183,3183,3183,...,3183,3183,3183,3183,3183,3183,3183,3183,3183,3183
yes,126,126,126,126,126,126,126,126,126,126,...,126,126,126,126,126,126,126,126,126,126


In [118]:
model['religious'].unique()

array(['unknown', 'yes'], dtype=object)

In [119]:
model['religious'].value_counts()

,juror_id,trial_id,race,gender,struck_by,strike_eligibility,no_responses,married,children,education,...,prior_info,death_hesitation,no_death,no_life,no_cops,yes_cops,legally_disqualified,witness_ambi,defendent_race,struck_by_prosecution
religious,,,,,,,,,,,,,,,,,,,,,
unknown,3228,3228,3228,3228,3228,3228,3228,3228,3228,3228,...,3228,3228,3228,3228,3228,3228,3228,3228,3228,3228
yes,125,125,125,125,125,125,125,125,125,125,...,125,125,125,125,125,125,125,125,125,125


In [120]:
model['education'].unique()

array(['unknown', 'attended graduate school', 'attended college',
       'college graduate'], dtype=object)

In [121]:
model['education'].value_counts()

,juror_id,trial_id,race,gender,struck_by,strike_eligibility,no_responses,married,children,religious,...,prior_info,death_hesitation,no_death,no_life,no_cops,yes_cops,legally_disqualified,witness_ambi,defendent_race,struck_by_prosecution
education,,,,,,,,,,,,,,,,,,,,,
attended college,7,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
attended graduate school,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
college graduate,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
unknown,3339,3339,3339,3339,3339,3339,3339,3339,3339,3339,...,3339,3339,3339,3339,3339,3339,3339,3339,3339,3339


In [235]:
model= model.replace({'race': {'Black': 1, 'White': 0}})

In [236]:
model = model.drop(columns = ['married', 'children', 'religious', 'education'])

In [237]:
model['strike_eligibility'].unique()

array(['n/a', 'Both State and Defense', 'State', 'Neither', 'Defense'],
      dtype=object)

In [131]:
model = model[model.strike_eligibility != 'n/a']

model = model[model.strike_eligibility != 'Neither']

model = model[model.strike_eligibility != 'Defense']

In [132]:
model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2216 entries, 2 to 3352
Data columns (total 70 columns):
juror_id                 2216 non-null int64
trial_id                 2216 non-null int64
race                     2216 non-null int64
gender                   2216 non-null object
struck_by                2216 non-null object
strike_eligibility       2216 non-null object
no_responses             2216 non-null bool
leans_state              2216 non-null bool
leans_defense            2216 non-null bool
leans_ambi               2216 non-null bool
moral_hardship           2216 non-null bool
job_hardship             2216 non-null bool
caretaker                2216 non-null bool
communication            2216 non-null bool
medical                  2216 non-null bool
employed                 2216 non-null bool
social                   2216 non-null bool
prior_jury               2216 non-null bool
crime_victim             2216 non-null bool
fam_crime_victim         2216 non-null bool
accu

In [133]:
model['struck_by_prosecution'].value_counts()

,juror_id,trial_id,race,gender,struck_by,strike_eligibility,no_responses,leans_state,leans_defense,leans_ambi,...,witness_state,prior_info,death_hesitation,no_death,no_life,no_cops,yes_cops,legally_disqualified,witness_ambi,defendent_race
struck_by_prosecution,,,,,,,,,,,,,,,,,,,,,
0,1664,1664,1664,1664,1664,1664,1664,1664,1664,1664,...,1664,1664,1664,1664,1664,1664,1664,1664,1664,1664
1,552,552,552,552,552,552,552,552,552,552,...,552,552,552,552,552,552,552,552,552,552


Class imbalance: 25% struck by pros, 75% move to defense

#### Feature Reduction

In [137]:
model = model.drop(columns=['juror_id', 'trial_id', 'strike_eligibility', 'struck_by' , 'no_responses'])

In [178]:
model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2216 entries, 2 to 3352
Data columns (total 66 columns):
race                     2216 non-null int64
gender                   2216 non-null object
leans_state              2216 non-null bool
leans_defense            2216 non-null bool
leans_ambi               2216 non-null bool
moral_hardship           2216 non-null bool
job_hardship             2216 non-null bool
caretaker                2216 non-null bool
communication            2216 non-null bool
medical                  2216 non-null bool
employed                 2216 non-null bool
social                   2216 non-null bool
prior_jury               2216 non-null bool
crime_victim             2216 non-null bool
fam_crime_victim         2216 non-null bool
accused                  2216 non-null bool
fam_accused              2216 non-null bool
eyewitness               2216 non-null bool
fam_eyewitness           2216 non-null bool
military                 2216 non-null bool
law_enforc

In [186]:
for column in model.columns[2:63]:
    print(model[column].value_counts())

False    2201
True       15
Name: leans_state, dtype: int64
False    2207
True        9
Name: leans_defense, dtype: int64
False    2198
True       18
Name: leans_ambi, dtype: int64
False    2214
True        2
Name: moral_hardship, dtype: int64
False    2204
True       12
Name: job_hardship, dtype: int64
False    2212
True        4
Name: caretaker, dtype: int64
False    2209
True        7
Name: communication, dtype: int64
False    2203
True       13
Name: medical, dtype: int64
False    2210
True        6
Name: employed, dtype: int64
False    2214
True        2
Name: social, dtype: int64
False    1960
True      256
Name: prior_jury, dtype: int64
False    2128
True       88
Name: crime_victim, dtype: int64
False    2111
True      105
Name: fam_crime_victim, dtype: int64
False    2183
True       33
Name: accused, dtype: int64
False    1951
True      265
Name: fam_accused, dtype: int64
False    2216
Name: eyewitness, dtype: int64
False    2216
Name: fam_eyewitness, dtype: int64
False    221

remove columns with very little variability (<21 in one class)

Stay: prior_jury, crime_victim, fam_crime_victim, accused, fam_accused, fam_law_enforcement, know_def, know_vic, know_wit, know_attny, prior_info, death_hesitation


In [188]:
model = model[['struck_by_prosecution','race', 'same_race','prior_jury', 'crime_victim', 'fam_crime_victim', 'accused', 'fam_accused', 'fam_law_enforcement', 'know_def', 
               'know_vic', 'know_wit', 'know_attny', 'prior_info', 'death_hesitation']]

In [189]:
model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2216 entries, 2 to 3352
Data columns (total 15 columns):
struck_by_prosecution    2216 non-null int64
race                     2216 non-null int64
same_race                2216 non-null int64
prior_jury               2216 non-null bool
crime_victim             2216 non-null bool
fam_crime_victim         2216 non-null bool
accused                  2216 non-null bool
fam_accused              2216 non-null bool
fam_law_enforcement      2216 non-null bool
know_def                 2216 non-null bool
know_vic                 2216 non-null bool
know_wit                 2216 non-null bool
know_attny               2216 non-null bool
prior_info               2216 non-null bool
death_hesitation         2216 non-null bool
dtypes: bool(12), int64(3)
memory usage: 95.2 KB


#### Add column

In [141]:
model['defendent_race'].unique()

array(['White', 'Black'], dtype=object)

In [142]:
model = model.replace({'defendent_race': {'Black': 1, 'White': 0}})

In [143]:
model['same_race'] = (model.race == model.defendent_race).astype(int)

In [190]:
model.head()

,struck_by_prosecution,race,same_race,prior_jury,crime_victim,fam_crime_victim,accused,fam_accused,fam_law_enforcement,know_def,know_vic,know_wit,know_attny,prior_info,death_hesitation
2,0,1,0,False,False,False,False,False,False,False,False,False,False,False,False
3,0,1,0,False,False,False,False,False,False,False,False,False,False,False,False
4,0,0,1,False,False,False,False,False,False,False,False,False,False,False,False
5,0,1,0,False,False,False,False,False,True,False,False,False,False,False,False
6,0,0,1,False,False,False,False,False,False,False,False,False,False,False,False


## Move data to modeling notebook

In [196]:
with open('model.pickle' , 'wb') as to_write:
    pickle.dump(model, to_write)

### Data for strike visualization

In [ ]:
visual = jurors[['race', 'struck_by', 'strike_eligibility']]

visual.info()

visual = visual[visual.race != 'Unknown']
visual = visual[visual.race != 'Latino']
visual = visual[visual.race != 'Asian']

visual.race.unique()

with open('visual.pickle' , 'wb') as to_write:
    pickle.dump(visual, to_write)